## PLAP
This is an implementation of the paper
Teng Huang, Bin-Bin Jia, Min-Ling Zhang. Progressive Label Propagation for Semi-Supervised Multi-Dimensional Classification. In: Proceedings of the 32th International Joint Conference on Artificial Intelligence (IJCAI'23), Macao, China.
***

### Requirement
- python == 3.10
- numpy == 1.21.5
- scikit-learn == 1.1.1
- scipy == 1.7.3
***

### Run
- Please make sure that you have installed numpy and sklearn package.
Possible commands: **conda install numpy** and **conda install scikit-learn**.

- Run all cells above **'Test'** for function definition.
- Run the first cell below **'Test'** for load data sets. Feel free to change data sets.
- Run the last cell for hamming score, exact match and sub-exact match obtained by **PLAP** method in *Song* data set with default parameter setting.

In [1]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import scipy.io as scio

Create SSL Dataset

In [2]:
def findFeasiableSet(X,Y):
    '''
    Find distinct indexes of examples which the corresponding union of label sets contains all labels.
    Input: 
    X: feature matrix.
    Y: label matrix.
    
    Output:
    A list of indexes.
    '''
    num_training = len(X)              #number of training examples
    num_dim = Y.shape[1]               #number of dimensions
    fea_set = []

    for dim in range(num_dim):
        labelset = list(set(Y[:,dim]))         
        for j in range(len(labelset)):
            for i in range(num_training):
                if Y[i][dim] == labelset[j]:
                    fea_set.append(i)
                    break
                
    return list(set(fea_set))

In [3]:
def createSSLData(X, Y, num_labeled,seed):   
    '''
    Create semi-supervised learning data set.
    Input:
    X: feature matrix with shape (m,d)
    Y: label matrix with shape (m,q)
    num_labeled: number of examples with known labels.
    seed: a seed for randomly shuffled data set in order to create different ssl data sets.
    
    Output:
    X: original feature matrix.
    Y: original label matrix.
    Y_ssl: generated semi-supervised learning label matrix.
    unlabeled_index: an ndarray of indexes of generated unlabeled samples.
    '''
    num_training = len(X)              #number of training examples
    fea_set = np.array(findFeasiableSet(X,Y))
    rng = np.random.RandomState(seed)
    index = np.arange(len(fea_set),num_training)
    rng.shuffle(index)
    index = np.hstack((fea_set,index))
    X = X[index]
    Y = Y[index]
    
    unlabeled_index = np.arange(num_labeled,num_training)
    Y_ssl = np.copy(Y)
    Y_ssl[unlabeled_index] = -1        #-1 reprents this sample is not labeled.
    
    return X,Y,Y_ssl,unlabeled_index

In [4]:
def getLabelNum(Y):
    '''
    Get the number of labels occured in label set w.r.t. each dimension.
    Input:
    Y: label matrix
    
    Output:
    an ndarray of which each element is the number of labels w.r.t. corresponding dimension.
    '''
    num_dim = Y.shape[1]               #number of dimensions
    labelnum = []
    for i in range(num_dim):
        num = len(set(Y[:,i]))         #number of different labels in each dimension
        labelnum.append(num)
        
    return np.array(labelnum)          #output: array style


def findSeeds(X,Y,num_labeled,num_seed):
    '''
    Find seeds which are able to be used for generating available data sets for semi-supervised learning, i.e., all labels occured in data sets should occur in training sets.
    Input:
    X: feature matrix with shape (m,d)
    Y: label matrix with shape (m,q)
    num_seed: number of seed needed. 

    Output:
    a list of available seeds
    '''
    seed_list = []
    sed = 0
    count = 0                           #number of appropriate seed 
    while count < num_seed:
        X,Y,Y_ssl,_ = createSSLData(X,Y,num_labeled=num_labeled,seed=sed)
        if (getLabelNum(Y) == getLabelNum(Y_ssl)-1).all():    #there is an extra label '-1' in Y_ssl
            count += 1
            seed_list.append(sed)
        sed += 1
    
    return seed_list

Step 1: Construct the label matrix $Y$

In [5]:
def getLabelMatrix(Y_ssl):
    '''
    Convert the ssl label matrix with real-valued elements into label matrix with binary elements.
    Input: 
    Y_ssl: ssl label set containing 1 or more dimensions.
    
    Output: label matrix corresponding to input label set.
    For multiple dimensions, output consists of different label matrixes stacked in 'diagonal blocks' (not used in fact).
    '''
    num_training = Y_ssl.shape[0]        #number of training examples
    num_dim = Y_ssl.shape[1]             #number of dimensions(class variables)
    label_per_dim = {}                   #class labels in each dimension
    num_per_dim = np.zeros((num_dim,1),dtype = int)  #number of class labels in each dimension

    for dim in range(num_dim):
        labelset = set(Y_ssl[:,dim])
        labelset.discard(-1)            #'-1' is not an actual label
        labelset.discard(0)
        label_per_dim[dim] = list(labelset)
        num_per_dim[dim] = len(label_per_dim[dim])

    for dim in range(num_dim):
        Y = np.zeros((num_training,int(num_per_dim[dim])))
        for p in range(num_training):
            for i in range(int(num_per_dim[dim])):
                if Y_ssl[p][dim] == label_per_dim[dim][i]:
                    Y[p][i] = 1
        if dim == 0:
            Labelmatrix = Y
        else:
            Labelmatrix = np.block([
                [Labelmatrix,np.zeros((Labelmatrix.shape[0],int(num_per_dim[dim])))],
                [np.zeros((num_training,Labelmatrix.shape[1])),Y]
                ])
            
    return Labelmatrix

Step 2: Construct the distance matrix $W$

In [6]:
def getDistanceMatrix(X,max_mat_elements=1000*1000):
    '''
    Compute distance matrix with euclidean distances between samples.
    Input: 
    X: feature matrix.
    max_mat_elements: maximum number of elements in matrixes computing at a time.
    
    Output:
    distance matrix.
    '''
    num_training = X.shape[0]      #number of training examples
    block_size = np.floor(max_mat_elements/num_training) #limit matrix storage to this much distance at a time
    num_blocks = np.ceil(num_training/block_size) #the number of times to compute
    dm = np.zeros((num_training,num_training))
    for iter in range(int(num_blocks)):
        low = int(block_size*iter)
        if iter == num_blocks:
            high = num_training
        else:
            high = int(block_size*(iter+1))

        temp_dm = euclidean_distances(X[low:high],X,squared=True)

        dm[low:high] = temp_dm
    
        
    return dm

Step 3: Calculate S such as  𝑆=𝐷−1/2𝑊𝐷−1/2

In [7]:
def getSimilarityMatrix(X,dm,n_neighbors=7,gamma=50):
    '''
    Define similarity matrix by Gaussian function.
    Input:
    X: feature matrix.
    dm: distance matrix.
    n_neighbors: number of nearest neighbors.
    gamma: parameter rbf kernel.
    
    Output:
    similarity matrix.
    '''
    num_feature = X.shape[1]
    if gamma is None:
        gamma = 1.0 / num_feature
    W = np.exp(-gamma*dm)
    np.fill_diagonal(W, 0)
        
    d = np.sum(W, axis=1)
    D = np.sqrt(d*d[:, np.newaxis])
    return np.divide(W,D,where=D!=0)

Step 4: Construct the P matrix $P$

In [8]:
def getPMatrix(Y_ssl,dm,unlabeled_index,n_neighbors):
    '''
    Statistics counted in neighborhood.
    Y_ssl: ssl label matrix.
    dm: distance matrix.
    unlabeled_index: indexes of unlabeled samples.
    n_neighbors: number of nearest neighbors.
    
    Output:
    a matrix which storages probability distribution of class labels within neighborhood.
    '''
    num_training = Y_ssl.shape[0]           #number of training examples
    num_dim = Y_ssl.shape[1]            #number of dimensions(class variables)
    label_per_dim = {}                  #class labels in each dimension
    num_per_dim = np.zeros((num_dim,1),dtype = int)  #number of class labels in each dimension
    for dim in range(num_dim):
        labelset = set(Y_ssl[:,dim])
        labelset.discard(-1)
        labelset.discard(0)
        label_per_dim[dim] = list(labelset)
        num_per_dim[dim] = len(label_per_dim[dim])
        
    np.fill_diagonal(dm,float("inf"))      #distance between instance and itself should be set as positive infinity
    labeled_index = np.arange(unlabeled_index[0])
    dm = dm[:, labeled_index]              #distance between unseen instance and training examples
    index_sorted = np.argsort(dm,axis=1) 
    
    if labeled_index.shape[0] < n_neighbors:
        n_neighbors = labeled_index.shape[0]
    neighbors = index_sorted[:,:n_neighbors]
    
    for dim in range(num_dim):
        P = np.zeros((num_training,int(num_per_dim[dim])))
        for i in range(num_training):
            for nei in neighbors[i]:
                for j in range(int(num_per_dim[dim])):
                    if Y_ssl[nei][dim] == label_per_dim[dim][j]:
                        P[i][j] += 1
        norms = np.linalg.norm(P, axis=1)      #normalization
        
        P = (P.T/norms).T

        if dim == 0:
            Pmatrix = P
        else:
            Pmatrix = np.block([
            [Pmatrix,np.zeros((Pmatrix.shape[0],int(num_per_dim[dim])))],
            [np.zeros((num_training,Pmatrix.shape[1])),P]
            ])
    

    return Pmatrix

Step 5: Update the progressive label matrix 

In [9]:
def getLabelMatrix_in_chain(Y_ssl,dm,label_per_dim,num_per_dim,unlabeled_index,current_dim,F_pre,n_neighbors):
    '''
    Get label matrix for dimensions subsequent to the first one.
    Input:
    Y_ssl: ssl label matrix.
    dm: distance matrix.
    label_per_dim: label set w.r.t. each dimention.
    num_per_dim: number of distinct labels in each dimention.
    unlabeled_index: indexes of unlabeled samples.
    current_dim: current dimension.
    F_pre: label matrix to be updated.
    n_neighbors: number of neighbors defined.
    
    Output: 
    a new label matrix.
    a mapping from previous matrix to current matrix.
    '''
    num_training = Y_ssl.shape[0]        #number of training examples
    # num_dim = Y_ssl.shape[1]             #number of dimensions(class variables)
    num_comb = F_pre.shape[1]            #number of combinations of label values from different dimensions that we already had
    num_current_dim = int(num_per_dim[current_dim]) #number of class labels in current dimension
    
    F = np.zeros((num_training,num_comb*num_current_dim))
    P = getPMatrix(Y_ssl[:,current_dim].reshape(-1,1),dm,unlabeled_index,n_neighbors=n_neighbors)
    mapping = np.arange(num_current_dim*num_comb).reshape(num_comb,num_current_dim)   #mapping from previous matrix to next matrix
    
    for p in range(num_training):
        if p not in unlabeled_index:     #training examples with labels
            for i in range(num_comb):    #0,1,…,k1-1
                for j in range (num_current_dim):  #0,1,…,k2-1
                    if F_pre[p][i] == 1 and Y_ssl[p][current_dim] == label_per_dim[current_dim][j]:
                        F[p][i*num_current_dim+j] = 1
                        
        elif p in unlabeled_index:       #training examples without labels
            for i in range(num_comb):    #0,1,…,k1-1
                for j in range (num_current_dim):  #0,1,…,k2-1
                    if F_pre[p][i] == 1:
                        F[p][i*num_current_dim+j] = P[p][j]
    
    return F,mapping

Step 6: Iterate label matrix until it converges

In [10]:
def propagation(Labelmatrix,Simmatrix,unlabeled_index,alpha = 0.99,max_iter = 1000,tol = 0.001):
    '''
    Label propagation for any label matrix.
    Input:
    Labelmatrix: label matrix needed to iteratively update.
    Simmatrix: simmilarity matrix.
    unlabeled_index: indexes of unlabeled samples.
    alpha: trade-off parameter.
    max_iter: maximum number of iteration.
    tol: convergence tolerance: threshold to consider the system at steady state.
    
    Output:
    result matrix.
    '''
    # num_training = Labelmatrix.shape[0]           #number of training examples
    labeled_index = np.arange(unlabeled_index[0]) 
    #The first iteration
    F = alpha*np.dot(Simmatrix, Labelmatrix) + (1-alpha)*Labelmatrix
    changed = np.abs(F-Labelmatrix).sum()
    
    for iter_num in range(max_iter):
        if changed < tol:
            break
            
        pre_F = F

        # propagation
        F = alpha*np.dot(Simmatrix, F) + (1-alpha)*Labelmatrix

        # check converge
        changed = np.abs(pre_F - F).sum() 
        
    if iter_num == max_iter and changed > tol:
        print('ConvergenceWarning')
    
    Labelmat_result = np.zeros_like(F)
    Labelmat_result[np.arange(len(F)), F.argmax(1)] = 1
    Labelmat_result[labeled_index] = Labelmatrix[labeled_index]
    
    return Labelmat_result

Evalution metrics:

In [11]:
def eva(Y,Y_result):
    '''
    Evaluations for MDC.
    '''
    num_testing = Y.shape[0]                     #number of training examples
    num_dim = Y.shape[1]                          #number of dimensions(class variables)
    num_correctdim = np.sum(Y == Y_result,axis=1)  #number of correct dimmensions for each example
        
    #Hamming Score(or Class Accuracy)
    HammingScore = np.sum(num_correctdim)/(num_dim*num_testing)    
    
    #Exact Match(or Example Accuracy or Subset Accuracy)
    ExactMatch = np.sum(num_correctdim == num_dim)/num_testing
    
    #Sub-ExactMatch    
    SubExactMatch = np.sum(num_correctdim >= num_dim-1)/num_testing

    return HammingScore,ExactMatch,SubExactMatch

In [12]:
def getResult_LGC(Y_ssl,Labelmat_result):
    '''
    Convert the label matrix w.r.t. single dimension with binary elements into result matrix with real-valued elements.
    '''
    num_testing = Y_ssl.shape[0]                      #number of training examples
    num_dim = Y_ssl.shape[1]                          #number of dimensions(class variables)
    label_per_dim = {}                                #class labels in each dimension
    num_per_dim = np.zeros((num_dim,1),dtype = int)   #number of class labels in each dimension
    
    count = 0
    num_dict = {}                                     #dict for accumulating vertical axis
    Y_result = np.zeros_like(Y_ssl)
    
    for dim in range(num_dim):
        labelset = set(Y_ssl[:,dim])
        labelset.discard(-1)            #'-1' is not a label
        labelset.discard(0)
        label_per_dim[dim] = list(labelset)
        num_per_dim[dim] = len(label_per_dim[dim])
        
    for dim in range(num_dim):
        num_dict[dim] = count
        count += int(num_per_dim[dim])
        

    for dim in range(num_dim):
        for i in range(num_testing):
            for j in range(int(num_per_dim[dim])):
                hori = dim*num_testing + i             #abscissa
                vert = num_dict[dim] + j               #ordinate
                if Labelmat_result[hori][vert] == 1:
                    Y_result[i][dim] = label_per_dim[dim][j]
                     
    return Y_result    

In [13]:
def getResult(label_per_dim,current_dim,Labelmat_result,mapping_chain):
    '''
    Get predicted labels from Labelmat_result matrix.
    Input:
    label_per_dim: label set w.r.t. each dimention.
    current_dim: current dimension considered.
    Labelmat_result: result matrix obtained from label propagation.
    mapping_chain: indexes mapping from previous matrix to current matrix.
    
    Output:
    result matrix with real number elements.
    '''
    num_training = Labelmat_result.shape[0]                     #number of training examples
    num_comb = Labelmat_result.shape[1]                         #number of combinations of label values from different dimensions
    Y_result = np.zeros((num_training,2))
  
    for i in range(num_training):
          for j in range(num_comb):
            if Labelmat_result[i][j] == 1:
                comb_previous,label_current_dim = np.argwhere(mapping_chain[current_dim]==j)[0]
                Y_result[i][1] = label_per_dim[current_dim][label_current_dim]
                continue
                     
    return Y_result    

In [14]:
def myLP(X,Y,Y_ssl,unlabeled_index,dm,Simmatrix,n_neighbors=7,alpha=0.99,max_iter=1000,tol=0.0001):
    '''
    PLAP method.
    Input:
    X: feature matrix.
    Y: label matrix.
    Y_ssl: ssl label matrix.
    unlabeled_index: indexes of unlabeled samples.
    dm: distance matrix.
    Simmatrix: simmilarity matrix.
    n_neighbors: number of nearest neighbors.
    alpha: trade-off parameter.
    max_iter: maximum number of iteration.
    tol: convergence tolerance: threshold to consider the system at steady state.
    
    Output:
    HammingScore,ExactMatch,SubExactMatch
    '''
    
    # num_training = X.shape[0]               #number of training examples
    num_dim = Y_ssl.shape[1]               #number of dimensions(class variables)
    label_per_dim = {}                  #class labels in each dimension
    num_per_dim = np.zeros((num_dim,1),dtype = int)   #number of class labels in each dimension
    for dim in range(num_dim):
        labelset = set(Y_ssl[:,dim])
        labelset.discard(-1)
        labelset.discard(0)
        label_per_dim[dim] = list(labelset)
        #label_per_dim[dim]=[int(i) for i in label_per_dim[dim]]
        num_per_dim[dim] = len(label_per_dim[dim])
        
    mapping_chain = {}    
    LabelMatrix = getLabelMatrix(Y_ssl[:,0].reshape(-1,1))   
    #Lable propagation procedure on the first dimension.
    F = propagation(LabelMatrix,Simmatrix,unlabeled_index,alpha=alpha,max_iter=max_iter,tol=tol)
    Y_result = getResult_LGC(Y_ssl[:,0].reshape(-1,1),F)
    Y_result_pre = Y_result

    #Progressive label propagation 
    for m in range(1,num_dim):
        LabelMatrix,mapping = getLabelMatrix_in_chain(Y_ssl,dm,label_per_dim,num_per_dim,unlabeled_index,current_dim=m,F_pre=F,n_neighbors=n_neighbors)
        F = propagation(LabelMatrix,Simmatrix,unlabeled_index,alpha=alpha,max_iter=max_iter,tol=tol)
        mapping_chain[m] = mapping
        Y_result = getResult(label_per_dim,m,F,mapping_chain)
        Y_result = np.hstack([Y_result_pre,Y_result[:,-1].reshape(-1,1)])
        Y_result_pre = Y_result

        if 2*F.nbytes+dm.nbytes+Simmatrix.nbytes>16e8:                    #memory limit (feel free to enrich it as long as enough memory available)
            F = getLabelMatrix(Y_result[:,-1].reshape(-1,1))              #back to one dimension due to the memory limit

    HammingScore,ExactMatch,SubExactMatch = eva(Y[:,:m+1][unlabeled_index],Y_result[unlabeled_index])
        # print("---> For examples with %d dimension(s), HammingScore:%f, ExactMatch:%f, SubExactMatch:%f" % \
              # (m+1, HammingScore, ExactMatch, SubExactMatch))
        
    return HammingScore,ExactMatch,SubExactMatch

## Test 

***
#### Default experimental Setting in PLAP
- 10 different randomly shuffled data sets are generated according to 10 seeds: seed $\in$ $\{0,1,2,\dots,9\}$
- number of labeled examples: $num\_ labeled \in \{40,50,60\}$
- rbf kernel: $gamma == 50$
- number of nearest neighbor: $n\_ neighbors == 7$
- trade-off parameter: $alpha == 0.99$

***
An example of PLAP on *Song* data set with 40 labeled examples and seed == 0.

The experimental results (mean $\pm$ std.) presented in formal paper are obtained by setting seed from 0 to 9.

In [15]:
data = scio.loadmat('dataset/Song')
X_train = np.array(data['data'][0][0][1])
y_train = np.array(data['target'],dtype = float)

In [16]:
X,Y,Y_ssl,unlabeled_index = createSSLData(X_train,y_train,num_labeled=40,seed=0)
dm = getDistanceMatrix(X,max_mat_elements=1000*1000)
S = getSimilarityMatrix(X,dm,gamma=50)
HammingScore,ExactMatch,SubExactMatch = myLP(X,Y,Y_ssl,unlabeled_index,dm,S,n_neighbors=7,alpha=0.99,max_iter=1000,tol=0.001)
print('Hamming Score == '+str(HammingScore))
print('Exact Match == '+str(ExactMatch))
print('Sub-Exact Match == '+str(SubExactMatch))

Hamming Score == 0.7409395973154362
Exact Match == 0.4
Sub-Exact Match == 0.836241610738255
